# Teste e prototipagem de parseador de CSV do formulário

In [20]:
import pandas as pd
import numpy as np
import json
import datetime as dt 
from PIL import Image, ImageOps

#import xavy.test as xx
import xavy.dataframes as xd

import clean as cc

* Ver se surgem nulls em datas preenchidas quando as padronizamos. Isso indica formatação estranha.

## Functions

In [21]:
def clean_form_data_v1(raw_df, tema_map, tipo_map, pais_map, inst2id, max_datasets=10):
    """
    Transforms raw data from CORDATA's form (a table read from 
    a CSV file) into a cleaned JSON, in the final Website format.
    
    Parameters
    ----------
    raw_df : DataFrame
        Data table read directly from the CSV file output by 
        CORDATA's form.
    tema_map: dict
        A str -> str dict that translates topics from Portuguese to
        Spanish.
    tipo_map: dict
        A str -> str dict that translates usecase types from 
        Portuguese to Spanish.
    pais_map: dict
        A str -> str dict that translates countries from Portuguese 
        to Spanish.
    max_datasets : int
        The maximum number of datasets allowed to be mentioned in 
        CORDATA's form.
    inst2id : defaultdict
        A mapper from institutions' names (authors) to IDs. If the 
        name is not found, `inst2id` should return None.
    
    Returns
    -------
    cleaned_json : dict
        The data in JSON structure, with metadata, ready to be
        used by CORDATA's website.
    """
    
    # Start cleaning process by copying data and creating an ID:
    cleaned_df = raw_df.copy()
    hash_ids = cc.build_hash_id(raw_df)
    cleaned_df.insert(0, 'hash_id', hash_ids)

    # Change NaN for None:
    cleaned_df = cleaned_df.replace({np.nan: None})

    # Standardize used dataset column names:
    cc.rename_first_data_cols(cleaned_df)

    # Rename columns:
    col_renamer = {'cobertura-geografica': 'geo_level', 'federatives': 'fed_units', 'cities': 'municipalities'}
    cleaned_df.rename(col_renamer, axis=1, inplace=True)
    
    # Basic cleaning of strings:
    str_cols = ['name', 'url', 'description', 'authors', 'countries', 'email', 'tags', 'url_source', 'url_image', 'comment', 'fed_units', 'municipalities']
    for c in str_cols:
        cleaned_df[c] = cleaned_df[c].str.strip()

    # Basic cleaning of strings associated to datasets:
    data_str_cols = ['data_name', 'data_institution', 'data_url']
    for i in range(1, max_datasets + 1):
        for c in data_str_cols:
            col = '{}_{}'.format(c, i)
            cleaned_df[col] = cleaned_df[col].str.strip()

    # Remove empty URLs (e.g. 'https://'): (not doing this because the website fails when some URLs are empty)
    url_cols = xd.sel_col_by_regex(cleaned_df, 'url')
    #cleaned_df.loc[:, url_cols] = cleaned_df.loc[:, url_cols].replace('^h?t?t?p?s?:?/?/?$', None, regex=True)
    
    # Fill missing source URL to avoid website failure:
    cleaned_df['url_source'].fillna('https://', inplace=True)    

    # Rename options in geo_level:
    geolevel_renamer = {'nao-se-aplica':'Não se aplica', 'mundial': 'Mundial', 'paises': 'Países', 'unidades-federativas':'Unidades federativas', 'municipios':'Municípios'}
    cleaned_df['geo_level'] = cleaned_df['geo_level'].map(geolevel_renamer)
    
    # Lowercase tags:
    cleaned_df['tags'] = cleaned_df['tags'].str.lower()

    # Split terms by semicolons:
    semicolon_cols = ['authors', 'email', 'tags']
    for c in semicolon_cols:
        cleaned_df[c] = cc.split_semicolons(cleaned_df[c])

    # Split terms by colons:
    colon_cols = ['countries', 'fed_units', 'municipalities']
    for c in colon_cols:
        cleaned_df[c] = cc.split_semicolons(cleaned_df[c], ',')

    # Parse usecase types and topics covered:
    assert set(cc.type_names)  == set(xd.sel_col_by_regex(cleaned_df, '^type_'))
    assert set(cc.topic_names) == set(xd.sel_col_by_regex(cleaned_df, '^topics_'))
    cleaned_df['type']   = cc.options_to_list(cleaned_df, cc.type_names)
    cleaned_df['topics'] = cc.options_to_list(cleaned_df, cc.topic_names)

    # Parse pub_date into month only:
    def parse_date(date):
        try:
            x = dt.datetime.strptime(date, '%d/%m/%Y')
        except:
            try:
                x = dt.datetime.strptime(date, '%m/%Y')
            except:
                x = dt.datetime.strptime(date, '%Y')
        return x.strftime('%m/%Y')
    cleaned_df['pub_date'] = cleaned_df['pub_date'].apply(parse_date)

    # Add country info for finer geographic levels:
    br_series = pd.Series('Brasil', index=cleaned_df.index).str.split()
    finer_geo = cleaned_df['geo_level'].isin({'Unidades federativas', 'Municípios'})
    cleaned_df.loc[finer_geo, 'countries'] = br_series.loc[finer_geo]
    cleaned_df.loc[finer_geo, 'countries_es'] = br_series.loc[finer_geo]

    # Add state info if municipalities is the level:
    mun_cov = cleaned_df['geo_level'] == 'Municípios'
    cleaned_df.loc[mun_cov, 'fed_units'] = cleaned_df.loc[mun_cov, 'municipalities'].str.join(', ').str.extractall('\(([A-Z]{2})\)').reset_index().groupby('level_0')[0].agg(lambda s: list(set(s)))
    
    # Parse datasets used by the usecases:
    data_cols = xd.sel_col_by_regex(cleaned_df, '^data_')
    data_df  = cleaned_df[data_cols]
    cleaned_df['datasets'] = data_df.apply(lambda row: cc.data_info_dict_list(cc.series2transposed_df(row), max_datasets), axis=1)

    # Change NaN for None:
    cleaned_df = cleaned_df.replace({np.nan: None})

    # Add processing date:
    cleaned_df['record_date'] = dt.datetime.today().strftime('%Y-%m-%d')
    
    # Update image URL (add generic image when none is provided):
    #cleaned_df['url_image'] = 'https://raw.githubusercontent.com/cewebbr/cordata/main/imagens/hash_id_' + cleaned_df['hash_id'].astype(str) + '.png'
    cleaned_df.loc[cleaned_df['url_image'] == 'https://', 'url_image'] = 'https://raw.githubusercontent.com/cewebbr/cordata/main/imagens/generic_usecase_banner.png'

    # Add inexistent columns:
    cleaned_df['modified_date'] = None
    cleaned_df['status'] = 'Em revisão'
    
    # Add author ID (from CGU's list):
    author_pos = int(np.arange(cleaned_df.shape[1])[cleaned_df.columns == 'authors'][0])
    cleaned_df.insert(author_pos + 1, 'authors_id', cleaned_df['authors'].apply(lambda l: cc.translate_list_elements(l, inst2id)))
    
    # Select relevant columns:
    trash_cols = xd.sel_col_by_regex(cleaned_df, r'type_|topics_|data_|continuar|_repeticao')
    good_cols  = [col for col in cleaned_df.columns if col not in trash_cols]
    cleaned_data = cleaned_df[good_cols].to_dict(orient='records')
    
    # Add translation to Spanish for categories:
    translated_data = [cc.add_translation(case_json, tipo_map, tema_map, pais_map) for case_json in cleaned_data]

    # Reverse order of data (so newest appear on the top):
    reversed_data = translated_data[::-1]

    # Add metadata (last update):
    metadata = {'last_update': dt.datetime.today().strftime('%Y-%m-%d')}
    cleaned_json = cc.embed_metadata(reversed_data, metadata)
    
    return cleaned_json

In [55]:
import requests
from io import BytesIO
from PIL import Image, ImageOps


def http_get(url, max_retries=3, timeout=10):
    """
    Make an HTTP GET request to `url` (str) and return a response object.

    Parameters
    ----------
    url : str
        The Web Address to request.
    max_retries : int
        Maximum number of request trials before giving up.
    timeout : float
        How long to wait for a response, in seconds.
    """
    # Prepare for request:
    session = requests.session()
    session.mount('http://', requests.adapters.HTTPAdapter(max_retries=max_retries))
    
    # GET:
    ssl_verify = True
    try:
        response = session.get(url, timeout=timeout)
    # In case of SSL Certificate error:
    except requests.exceptions.SSLError:
        ssl_verify = False
        response = session.get(url, timeout=timeout, verify=ssl_verify)

    # If forbidden, try pretending to be a browser:
    if response.status_code in {403, 404, 406, 412, 503, 429}:
        browser_header = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:136.0) Gecko/20100101 Firefox/136.0'}
        response = session.get(url, timeout=timeout, headers=browser_header, verify=ssl_verify)
    
    # If fail:
    if response.status_code != 200:
        raise Exception('HTTP request failed with code ' + str(response.status_code))

    return response

    
def standardize_image_from_url(url, target_height, target_width, output_path):
    """
    Downloads an image from the Web and standardizes it to a target aspect ratio and size.

    Steps:
    1. Downloads JPG, PNG, or GIF image from URL.
    2. Computes current aspect ratio.
    3. Crops or pads symmetrically based on deviation from target aspect ratio.
    4. Scales image to target width while maintaining aspect ratio.
    5. Saves output as PNG.

    Parameters:
        url (str): URL of the image.
        target_height (int): Target height in pixels.
        target_width (int): Target width in pixels.
        output_path (str): Path to save transformed PNG image.
    """
    # Step 1: Download image from URL
    response = http_get(url)
    response.raise_for_status()  # Raises error for invalid response
    if response.headers.get("Content-Type")[:9] != 'text/html':
        #print(response.headers.get("Content-Type"))
        img = Image.open(BytesIO(response.content)).convert("RGB")
    
        # Compute image and target aspect ratios
        width, height = img.size
        aspect = width / height
        target_aspect = target_width / target_height
    
        # Step 3: If aspect ratio is more than twice the target, crop horizontally
        if aspect > 2 * target_aspect:
            new_width = int(height * target_aspect)
            left = (width - new_width) // 2
            right = left + new_width
            img = img.crop((left, 0, right, height))
            width, height = img.size
            aspect = width / height
    
        # Step 4: If aspect ratio is less than half the target, crop vertically
        elif aspect < 0.5 * target_aspect:
            new_height = int(width / target_aspect)
            top = (height - new_height) // 2
            bottom = top + new_height
            img = img.crop((0, top, width, bottom))
            width, height = img.size
            aspect = width / height
    
        # Step 5: If slightly wider, add gray bands on top/bottom
        elif aspect > target_aspect:
            new_height = int(width / target_aspect)
            padding = (new_height - height) // 2
            img = ImageOps.expand(img, border=(0, padding, 0, padding), fill=(128, 128, 128))
            width, height = img.size
    
        # Step 6: If slightly narrower, add gray bands on left/right
        elif aspect < target_aspect:
            new_width = int(height * target_aspect)
            padding = (new_width - width) // 2
            img = ImageOps.expand(img, border=(padding, 0, padding, 0), fill=(128, 128, 128))
            width, height = img.size
    
        # Step 7: Scale image to have target width (keeping aspect)
        new_height = int((target_width / width) * height)
        img = img.resize((target_width, new_height), Image.LANCZOS)
    
        # Step 8: Save as PNG
        img.save(output_path, format="PNG")

        return True
        
    else:
        return False
        
# Example usage:
# standardize_image_from_url("https://example.com/image.jpg", 600, 800, "output.png")

In [52]:
def filter_usecases(data_json, field, value):
    """
    Select from the data (including the metadata) `data_json`
    only the usecases whose `field` (str) exactly matches `value`
    or that is included in `value` (list, set, Series, etc).
    
    Returns a list.
    """
    
    if type(value) not in {list, set, tuple, pd.Series, np.ndarray}:
        value = {value}
    if type(value) == pd.Series:
        value = value.to_list()
    if type(value) == np.ndarray:
        value = value.tolist()    
    
    data = data_json['data']
    sel = list(filter(lambda usecase: usecase[field] in value, data))
    return sel

## Translations

In [24]:
# Load translations:
translation_df = pd.read_excel('../docs/conteudo_site/textos_site_traducao_traduzidos_v05.xlsx', sheet_name='Site')

# Build translators:
tema_map = cc.get_translator(translation_df, r'filtro_tema_\d')
tipo_map = cc.get_translator(translation_df, r'filtro_tipo_\d', lower=True)
pais_map = cc.get_translator(translation_df, r'filtro_pais_\d')

In [25]:
# Mapper from Institution's name to CGU ID: 
inst2id = cc.load_institution_identifier('../dados/aux/organizacao_*.json')

## Loading data

In [26]:
#raw_df = pd.read_csv('../dados/testes/cordata_formulario/20230825081146_dados-abertos.csv', sep=';')
#raw_df = pd.read_csv('../dados/testes/cordata_formulario/20230904055149_dados-abertos.csv', sep=';')
#raw_df = pd.read_csv('../dados/testes/cordata_formulario/20230914064259_dados-abertos.csv', sep=';')
#raw_df = pd.read_csv('../dados/testes/cordata_formulario/20230920091624_dados-abertos.csv', sep=';')
#raw_df = pd.read_csv('../dados/testes/cordata_formulario/20231025093912_dados-abertos.csv', sep=';')
#raw_df = pd.read_csv('../dados/testes/cordata_formulario/20231025104350_dados-abertos.csv', sep=';')
#raw_df = pd.read_csv('../dados/testes/cordata_formulario/20231027103427_dados-abertos.csv', sep=';')
#raw_df = pd.read_csv('../dados/testes/cordata_formulario/20231027071604_dados-abertos.csv', sep=';')
#raw_df = pd.read_csv('../dados/curados/20240207112512_dados-abertos.csv', sep=';')
#raw_df = pd.read_csv('../dados/testes/cordata_formulario/20240215060030_dados-abertos.csv', sep=';')
#raw_df = pd.read_csv('../dados/brutos/20240301082824_dados-abertos.csv', sep=';')
#raw_df = pd.read_csv('../dados/brutos/20240415125934_dados-abertos.csv', sep=';')
#raw_df = pd.read_csv('../dados/curados/20240828043115_dados-abertos.csv', sep=';')
#raw_df = pd.read_csv('../dados/curados/20241021052054_dados-abertos.csv', sep=';')
#raw_df = pd.read_csv('../dados/brutos/20241022065612_dados-abertos.csv', sep=';')
#raw_df = pd.read_csv('../dados/brutos/20251013044302_dados-abertos.csv', sep=';')
#raw_df = pd.read_csv('../dados/brutos/20251017073050_dados-abertos.csv', sep=';')
raw_df = pd.read_csv('../dados/curados/20251017073050_dados-abertos.csv', sep=';')

In [27]:
# Load previously saved data:
#previous_json = cc.load_json('../dados/limpos/usecases_current.json')
previous_json = cc.load_json('../dados/limpos/usecases_n28_2025-10-13.json')

In [28]:
# Fix previous structure:
previous_data = previous_json['data']
for d in previous_data:
    d['geo_level'] = None
    d['fed_units'] = None
    d['municipalities'] = None
    d['modified_date'] = None
    d['status'] = 'Em revisão'

## Cleaning and preparing the data

In [29]:
# Select entries to include in the data JSON:
# Those not present already:
previous_urls = [uc['url'] for uc in previous_json['data']]
sel_df = raw_df.loc[~raw_df['url'].isin(previous_urls)]

In [23]:
# Clean data:
wmeta_json = clean_form_data_v1(sel_df, tema_map, tipo_map, pais_map, inst2id)

In [24]:
# Join old data with new data:
updated_json = {'metadata': wmeta_json['metadata'], 
                'data': wmeta_json['data'] + previous_json['data']}

In [25]:
print('RENOMEAR AS IMAGENS')
rename_imgs = {uc['name']: uc['url_image'].split('/')[-1] for uc in filter_usecases(updated_json, 'url', sel_df['url'])}
for n, i in rename_imgs.items():
    print(n, '==>', i)

RENOMEAR AS IMAGENS
Tataravô de Fernando Henrique Cardoso teria usado escravizados em corrida por ouro ==> Capa_Tataravo-de-Fernando-Henrique-Cardoso-teria-usado-escravizados-em-corrida-por-ouro.png
Por que é tão difícil traçar a genealogia de pessoas negras no Brasil? ==> CAPA-Por-que-e-tao-dificil-tracar-a-genealogia-de-pessoas-negras-no-Brasil.jpg
Vendem-se Marias: os indícios de escravidão na linhagem de Itamar Franco ==> Capa-Vendem-se-Marias-os-indicios-de-escravidao-na-linhagem-de-Itamar-Franco.png
Movimento pela reparação histórica da escravidão ganha força no Brasil e no mundo ==> Capa-Movimento-pela-reparacao-historica-da-escravidao-ganha-forca-no-Brasil-e-no-mundo.png
Herói da independência do Piauí é quinto avô de Sarney e teria passado escravizador ==> Capa-Heroi-da-independencia-do-Piaui-e-quinto-avo-de-Sarney-e-teria-passado-escravizador.jpg
Famílias que teriam origem escravocrata mantêm poder há 200 anos ==> Capa-Clas-familiares-com-origem-escravocrata-mantem-poder-ha-2

In [26]:
# Save to file:
if False:
    n_usecases = len(updated_json['data'])
    today = dt.datetime.today().strftime('%Y-%m-%d')
    with open('../dados/limpos/usecases_n{}_{}.json'.format(n_usecases, today), 'w') as f:
        json.dump(updated_json, f, ensure_ascii=False, indent=2)
#    with open('../dados/limpos/usecases_current.json'.format(n_usecases, today), 'w') as f:
#        json.dump(updated_json, f, ensure_ascii=False, indent=2)

## Particular adjustments made to the data

### Download and standardize images

In [58]:
hash_ids = cc.build_hash_id(sel_df)
df = pd.DataFrame({'id': hash_ids, 'name': sel_df['name'], 'image':sel_df['url_image']})
df = df.loc[df['image'].str.len() > 8]
#dataset = []
for i in range(25, len(df)):
    c = df.iloc[i]
    print(i, c['id'], c['image'], end=' >> ')
    outfile = '../imagens/hash_id_{:}.png'.format(c['id'])
    success = standardize_image_from_url(c['image'], 293, 523, outfile)
    print(success)
    if success == True:
        image = f"https://raw.githubusercontent.com/cewebbr/cordata/main/imagens/hash_id_{c['id']}.png"
    else:
        image = c['image']
    
    data = {'id': c['id'], 'name': c['name'], 'url_image': image}
    dataset.append(data)

25 1998242644 https://www.piauinegocios.com.br/controle/arquivos/images/eolica-lagoa-dos-ventos-enel-green-power-brasil.jpeg >> True
26 2437576788 https://s2-g1.glbimg.com/t551LIgRM3DvAxjIepGbdask5IU=/0x0:651x3209/984x0/smart/filters:strip_icc()/i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538a/internal_photos/bs/2025/S/G/8qPyDeTv6QMA5qmVY4Sw/g1-arte-vitimas-expostas-v3.jpg >> True
27 1263137159 https://s2-g1.glbimg.com/l8sWvYaVdCaXI3dIllBpohQkmCQ=/0x0:1615x901/984x0/smart/filters:strip_icc()/i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538a/internal_photos/bs/2025/B/6/Q1iXp6QEK9bYYEtfos4Q/screenshot-1.jpg >> True
28 4045053834 https://opara.nyc3.cdn.digitaloceanspaces.com/ojoio/uploads/2024/10/corpo-portfolio-produtos.jpg >> True
29 3146776786 https://opara.nyc3.cdn.digitaloceanspaces.com/ojoio/uploads/2025/02/Foto_Fazenda-Sao-Jorge.jpg >> True
30 1123926844 https://opara.nyc3.cdn.digitaloceanspaces.com/ojoio/uploads/2025/03/evolucao-patrimonio-fiagro-cra-v2.png >> True


/home/hxavier/ceweb/projetos/cordata/codigo/env/lib/python3.8/site-packages/PIL/Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


True
33 656486850 http://agenciasn.com.br/wp-content/uploads/2015/11/Chuva2015Out_0018xxxxx-660x330.jpg >> True
34 871299308 https://tribunademinas.com.br/colunas/dinheiroecidadania/wp-content/uploads/2025/10/beneficiarios-direito-adicionais-bolsa-familia-750x375.webp >> True
35 2608517022 https://cfn.org.br/wp-content/uploads/2015/09/logo_cfn_sized-770x214.png >> True
36 3082641303 https://opara.nyc3.cdn.digitaloceanspaces.com/ojoio/uploads/2024/12/aquifero-Caue-01-1536x1208.jpg >> True
37 4232934708 https://upload.wikimedia.org/wikipedia/commons/thumb/d/d3/Rocha_Le%C3%A3o%2C_o_Bar%C3%A3o_de_Itamarati_-_Rio_de_Janeiro_-_20230919053747.jpg/2560px-Rocha_Le%C3%A3o%2C_o_Bar%C3%A3o_de_Itamarati_-_Rio_de_Janeiro_-_20230919053747.jpg >> True
38 3913815942 https://s2-g1.glbimg.com/XviO039iHUaNoQoymRD6_cWhhJ0=/0x0:1325x724/984x0/smart/filters:strip_icc()/i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538a/internal_photos/bs/2024/m/l/f3BDdvSAawfRJUryP0FA/fazenda-remigio1.jpeg >> True
39 84

In [63]:
dataset_df = pd.DataFrame(dataset)
#dataset_df.to_csv('/home/hxavier/temp/new_cordata_images.csv', index=False)

In [68]:
new_images_df = dataset_df.loc[dataset_df['url_image'].str.contains('cewebbr')]

In [64]:
with open('../dados/limpos/usecases_current.json', 'r') as f:
    usecases = json.load(f)

In [71]:
for i in range(len(new_images_df)):
    x = new_images_df.iloc[i]
    uc = list(filter(lambda z: z['hash_id'] == x['id'], usecases['data']))[0]
    uc['url_image'] = x['url_image']

In [73]:
usecases['metadata']['last_update'] = '2025-10-20'

In [77]:
#with open('../dados/limpos/usecases_n109_2025-10-20.json', 'w') as f:
#    json.dump(usecases, f, ensure_ascii=False, indent=1)

### Removing first and last cases

In [32]:
# Remove first and last usecases (sensible cases):
#previous_json['data'] = previous_json['data'][1:-1]

In [33]:
# Save to file:
if False:
    with open('../dados/limpos/usecases_n10_2024-01-03.json', 'w') as f:
        json.dump(previous_json, f, ensure_ascii=False, indent=2)

### Adding CGU institutions' IDs

In [110]:
cleaned_data = previous_json['data']

for usecase in cleaned_data:
    usecase['authors_id'] = [inst2id[a] for a in usecase['authors']] 

In [112]:
#with open('../dados/limpos/test.json', 'w') as f:
#    json.dump(previous_json, f, ensure_ascii=False, indent=2)

### Renaming topics to standardize with CGU

In [41]:
from xavy.utils import translate_dict

In [42]:
cordata2cgu = {'Agropecuária':'Agricultura, extrativismo e pesca',
               'Administração Pública':'Governo e Política',
               'Alimentação':'Assistência e Desenvolvimento Social',
               'Arte e Cultura':'Cultura, Lazer e Esporte',
               'Atividade industrial, de comércio ou serviços':'Comércio, Serviços e Turismo',
               'Ciência e Tecnologia':'Ciência, Informação e Comunicação',
               'Comunicações':'Ciência, Informação e Comunicação',
               'Consumo':'Economia e Finanças',
               'Dados demográficos':'Dados Estratégicos',
               'Defesa':'Defesa e Segurança',
               'Direito e Processual Penal':'Justiça e Legislação',
               'Direitos Humanos':'Assistência e Desenvolvimento Social',
               'Economia':'Economia e Finanças',
               'Esporte':'Cultura, Lazer e Esporte',
               'Finanças e Orçamento Público':'Economia e Finanças',
               'Imóveis, Habitação e Urbanismo':'Habitação, Saneamento e Urbanismo',
               'Justiça e Direito':'Justiça e Legislação',
               'Lazer':'Cultura, Lazer e Esporte',
               'Multimídia':'Cultura, Lazer e Esporte',
               'Política, Partidos e Eleições':'Governo e Política',
               'Política, Partidos e Eleições': 'Governo e Política',
                'Previdência e Assistência Social': 'Assistência e Desenvolvimento Social',
                'Processo Legislativo': 'Governo e Política',
                'Recursos hídricos': 'Meio Ambiente',
                'Recursos minerais': 'Economia e Finanças',
                'Redes Sociais': 'Ciência, Informação e Comunicação',
                'Religião': 'Cultura, Lazer e Esporte',
                'Segurança': 'Defesa e Segurança',
                'Terras': 'Geografia',
                'Trabalho e Emprego': 'Trabalho',
                'Transparência': 'Governo e Política',
                'Transporte': 'Transportes e Trânsito',
                'Turismo': 'Comércio, Serviços e Turismo'
               }
cordata2cgu = translate_dict(cordata2cgu)

def cgu_topic(tlist):
    """
    Translate a list of v1 CORDATA topics to v1 CGU topics.
    """
    return sorted(list(set([cordata2cgu[t] for t in tlist])))

In [49]:
# Translate topics in Portuguese: 
data = previous_json['data']
for usecase in data:
    usecase['topics'] = cgu_topic(usecase['topics'])

In [75]:
for case_json in data:
    cc.add_translation(case_json, tipo_map, tema_map, pais_map)

In [ ]:
# Save to file:
if False:
    with open('../dados/limpos/usecases_n12_2024-01-02.json', 'w') as f:
        json.dump(previous_json, f, ensure_ascii=False, indent=2)

### Reversing data order

In [3]:
# Load previously saved data:
if True:
    with open('../dados/limpos/usecases_n12_2023-10-27.json', 'r') as f:
        updated_json = json.load(f)

In [8]:
# Reverse order:
updated_json['data'] = updated_json['data'][::-1]

In [11]:
# Remove last usecase:
updated_json['data'] = updated_json['data'][:-1]

In [17]:
# Remove first usecase:
updated_json['data'] = updated_json['data'][1:]

In [20]:
# Update last update data:
updated_json['metadata']['last_update'] = '2023-10-31'

# Trash

## Filtering data

In [29]:
filter_usecases(previous_json, 'url_source', None)

[{'hash_id': 1092493666,
  'name': 'Aplicativo Democracia Beta',
  'url': 'https://onelink.to/bgmk2n',
  'description': 'Desenvolvemos o app Democracia Beta para tornar a participação do cidadão na política fácil e objetiva. O aplicativo coleta informações de fontes públicas e oficiais para apresentar as atividades de Deputados Federais, Senadores e do Presidente da República, possibilitando que o cidadão acompanhe e fiscalize os seus Representantes. \n\nInteligência Artificial (ChatPDF) ajuda a entender melhor Projetos de Lei e Propostas de Emenda à Constituição, levantando prós e contras da Proposição. E o usuário pode expressar sua opinião nas atividades dos seus Representantes, escolhendo entre "gostei",  "não gostei" ou "me abstenho", e apenas ele terá acesso a suas manifestações. Para garantir um posicionamento mais isento, o político em questão permanece oculto antes da escolha, sendo revelado assim que a decisão é tomada, juntamente com o resultado geral anônimo dos posicioname

## Criando nomes das colunas de temas no CSV

In [8]:
df = pd.read_csv('/home/hxavier/temp/Campos do catálogo de reúso de dados abertos - Temas dos dados.csv', 
                 names=['topic', 'tag'])

In [9]:
from xavy.text import remove_accents

In [10]:
df['tag'] = 'topics_' + df['tag'].apply(remove_accents)

In [11]:
df.set_index('tag')['topic'].to_dict()

{'topics_agricultura_extrativismo_pesca': 'Agricultura, extrativismo e pesca',
 'topics_assistencia_desenvolvimento_social': 'Assistência e Desenvolvimento Social',
 'topics_ciencia_informacao_comunicacao': 'Ciência, Informação e Comunicação',
 'topics_comercio_servicos_turismo': 'Comércio, Serviços e Turismo',
 'topics_cultura_lazer_esporte': 'Cultura, Lazer e Esporte',
 'topics_dados_estrategicos': 'Dados Estratégicos',
 'topics_defesa_seguranca': 'Defesa e Segurança',
 'topics_economia_financas': 'Economia e Finanças',
 'topics_educacao': 'Educação',
 'topics_energia': 'Energia',
 'topics_equipamentos_publicos': 'Equipamentos Públicos',
 'topics_geografia': 'Geografia',
 'topics_governo_politica': 'Governo e Política',
 'topics_habitacao_saneamento_urbanismo': 'Habitação, Saneamento e Urbanismo',
 'topics_industria': 'Indústria',
 'topics_justica_legislacao': 'Justiça e Legislação',
 'topics_meio_ambiente': 'Meio Ambiente',
 'topics_plano_plurianual': 'Plano Plurianual',
 'topics_re

In [21]:
df['tag'] = 'topics_' + df['tag'].apply(remove_accents)

In [23]:
#df.to_csv('/home/hxavier/temp/topics_tags.csv', index=False)

In [33]:
tab_df = pd.read_csv('/home/hxavier/temp/tabela_tema_tag.csv')

In [41]:
dict(zip(list(cc.topic_names.keys())[:22], tab_df['Tema'].iloc[:22]))

{'topics_agropecuaria': 'Agricultura, extrativismo e pesca',
 'topics_alimentacao': 'Assistência e Desenvolvimento Social',
 'topics_administracao_publica': 'Ciência, Informação e Comunicação',
 'topics_arte_e_cultura': 'Comércio, Serviços e Turismo',
 'topics_atividade_industrial_de_comercio_ou_servicos': 'Cultura, Lazer e Esporte',
 'topics_ciencia_e_tecnologia': 'Dados Estratégicos',
 'topics_comunicacoes': 'Defesa e Segurança',
 'topics_consumo': 'Economia e Finanças',
 'topics_dados_demograficos': 'Educação',
 'topics_defesa': 'Energia',
 'topics_direito_e_processual_penal': 'Equipamentos Públicos',
 'topics_direitos_humanos': 'Geografia',
 'topics_economia': 'Governo e Política',
 'topics_educacao': 'Habitação, Saneamento e Urbanismo',
 'topics_energia': 'Indústria',
 'topics_esporte': 'Justiça e Legislação',
 'topics_financas_e_orcamento_publico': 'Meio Ambiente',
 'topics_imoveis_habitacao_e_urbanismo': 'Plano Plurianual',
 'topics_justica_e_direito': 'Relações Internacionais',

0        Agricultura, extrativismo e pesca
1     Assistência e Desenvolvimento Social
2        Ciência, Informação e Comunicação
3             Comércio, Serviços e Turismo
4                 Cultura, Lazer e Esporte
5                       Dados Estratégicos
6                       Defesa e Segurança
7                      Economia e Finanças
8                                 Educação
9                                  Energia
10                   Equipamentos Públicos
11                               Geografia
12                      Governo e Política
13       Habitação, Saneamento e Urbanismo
14                               Indústria
15                    Justiça e Legislação
16                           Meio Ambiente
17                        Plano Plurianual
18                 Relações Internacionais
19                                   Saúde
20                                Trabalho
21                  Transportes e Trânsito
Name: Tema, dtype: object

## Treating new data after topics change

In [24]:
raw_df = pd.read_csv('../dados/testes/cordata_formulario/20240207112512_dados-abertos.csv', sep=';')

In [32]:
clean_form_data_v1(raw_df, tema_map, tipo_map, pais_map, inst2id)

TypeError: 'NoneType' object is not iterable

## Enriching data

In [37]:
institutions = set(cleaned_df[xd.sel_col_by_regex(cleaned_df, 'data_institution_')].values.flatten())
institutions

{'CGU',
 'Câmara dos deputados',
 'Ministério da Economia',
 None,
 'Prefeitura do Município de São Paulo',
 'SEADE',
 'Senado Federal',
 'TSE'}

## Visualizing cleaned data

### JSON

In [129]:
updated_json['data'][3]

{'hash_id': 1191967625,
 'name': 'Tutorial de dados',
 'url': 'https://github.com/tecMTST/tutorial-de-dados',
 'description': 'Pequenos projetos de análise de dados desenvolvidos em notebooks de Python que podem ser seguidos como tutoriais.',
 'pub_date': '09/2023',
 'authors': ['Núcleo de Tecnologia do MTST'],
 'countries': ['Brasil'],
 'email': ['mtst@nucleodetecnologia.com.br'],
 'tags': ['tutorial',
  'análise de dados',
  'ciência de dados',
  'expectativa de vida',
  'financiamento de campanha',
  'financiamento eleitoral',
  'desigualdade',
  'são paulo'],
 'url_source': 'https://github.com/tecMTST/tutorial-de-dados',
 'url_image': 'https://nucleodetecnologia.com.br/assets/img/novo-logo-tecnologia.svg',
 'comment': None,
 'type': ['material didático', 'estudo independente'],
 'topics': ['Direitos Humanos',
  'Imóveis, Habitação e Urbanismo',
  'Política, Partidos e Eleições',
  'Saúde'],
 'datasets': [{'data_name': 'Mortalidade no município de Sâo Paulo',
   'data_institution': 

### CSV

In [8]:
s_cols = 9
n_cols = cleaned_df.shape[-1]
for i in range(n_cols // s_cols + 1):
    display(cleaned_df.iloc[:, i * s_cols:(i + 1) * s_cols])

,name,url,description,pub_date,authors,countries,email,type_artigo,type_materia_jornalistica
0,Três partidos do Centrão controlam mais de R$ ...,https://oglobo.globo.com/politica/tres-partido...,Matéria jornalística publicada no Jornal O Glo...,01/2022,"[Dimitrius Dantas, Daniel Gullino, Bruno Góes]",[Brasil],"[bruno.goes@oglobo.com.br, daniel.gullino@bsb....",0,1
1,Serenata de amor,https://serenata.ai,Um projeto aberto que usa ciência de dados com...,06/2016,"[Open Knowledge Brasil, Irio Musskopf]",[Brasil],[contato@serenata.ai],0,0
2,Base dos Dados,https://basedosdados.org/,Catálogo de conjuntos de dados com alguns conj...,09/2020,[Instituto Base dos Dados],"[Mundial, Brasil]",NaN,0,0
3,Tutorial de dados,https://github.com/tecMTST/tutorial-de-dados,Pequenos projetos de análise de dados desenvol...,09/2023,[Núcleo de Tecnologia do MTST],[Brasil],[mtst@nucleodetecnologia.com.br],0,0


,type_aplicativo_plataforma,type_painel,type_conjunto_de_dados,type_material,type_estudo,type_bot,type_outro,topics_agropecuaria,topics_alimentacao
0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,1,0,0,0
2,0,0,1,0,0,0,0,1,1
3,0,0,0,1,1,0,0,0,0


,topics_administracao_publica,topics_arte_e_cultura,topics_atividade_industrial_de_comercio_ou_servicos,topics_ciencia_e_tecnologia,topics_comunicacoes,topics_consumo,topics_dados_demograficos,topics_defesa,topics_direito_e_processual_penal
0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1
3,0,0,0,0,0,0,0,0,0


,topics_direitos_humanos,topics_economia,topics_educacao,topics_energia,topics_esporte,topics_financas_e_orcamento_publico,topics_imoveis_habitacao_e_urbanismo,topics_justica_e_direito,topics_lazer
0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,1,0,0,0
2,1,1,1,1,1,1,1,1,1
3,1,0,0,0,0,0,1,0,0


,topics_meio_ambiente,topics_multimidia,topics_politica_partidos_e_eleicoes,topics_previdencia_e_assistencia_social,topics_recursos_hidricos,topics_recursos_minerais,topics_redes_sociais,topics_relacoes_internacionais,topics_religiao
0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1
3,0,0,1,0,0,0,0,0,0


,topics_saude,topics_seguranca,topics_terras,topics_transporte,topics_transparencia,topics_trabalho_e_emprego,topics_turismo,tags,url_source
0,0,0,0,0,0,0,0,"[orçamento federal, orçamento secreto, bolsona...",https://github.com/gabinete-compartilhado-acre...
1,0,0,0,0,1,0,0,"[ceap, Cota para Exercício da Atividade Parlam...",https://github.com/okfn-brasil/serenata-de-amor
2,1,1,1,1,1,1,1,"[catálogo, repositório]",https://github.com/basedosdados
3,1,0,0,0,0,0,0,"[tutorial, análise de dados, ciência de dados,...",https://github.com/tecMTST/tutorial-de-dados


,url_image,data_name_1,data_institution_1,data_url_1,data_periodical_1,continuar,data_name_2,data_institution_2,data_url_2
0,https://ogimg.infoglobo.com.br/in/25354165-974...,SIGA Brasil,Senado Federal,https://www12.senado.leg.br/orcamento/sigabrasil,2,1,Orçamento da despesa pública,CGU,https://portaldatransparencia.gov.br/orcamento...
1,https://d33wubrfki0l68.cloudfront.net/44a83a26...,Cota parlamentar,Câmara dos deputados,https://www.camara.leg.br/cota-parlamentar/,1,1,Informações sobre deputados federais,Câmara dos deputados,https://www.camara.leg.br/SitCamaraWS/Deputado...
2,https://avatars.githubusercontent.com/u/710976...,NaN,NaN,NaN,1,2,NaN,NaN,NaN
3,https://nucleodetecnologia.com.br/assets/img/n...,Mortalidade no município de Sâo Paulo,Prefeitura do Município de São Paulo,https://www.prefeitura.sp.gov.br/cidade/secret...,2,1,"População do município de São Paulo, por idade",SEADE,https://repositorio.seade.gov.br/group/seade-p...


,data_periodical_2,segunda_repeticao,data_name_3,data_institution_3,data_url_3,data_periodical_3,terceira_repeticao,data_name_4,data_institution_4
0,2.0,1.0,Plataforma +Brasil,Ministério da Economia,https://antigo.plataformamaisbrasil.gov.br/dow...,2.0,2.0,NaN,NaN
1,1.0,1.0,API de Compras Governamentais,Ministério da Economia,http://compras.dados.gov.br,1.0,2.0,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.0,1.0,Fronteiras dos distritos do município de São P...,Prefeitura do Município de São Paulo,https://geosampa.prefeitura.sp.gov.br,2.0,1.0,Prestação de contas eleitorais,TSE


,data_url_4,data_periodical_4,quarta_repeticao,data_name_5,data_institution_5,data_url_5,data_periodical_5,comment,type
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[matéria jornalística]
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[painel, dashboard ou infográfico, bot]"
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"São muitas bases de dados, não seria possível ...",[conjunto de dados]
3,https://dadosabertos.tse.jus.br/dataset/,2.0,1.0,Candidatos eleitorais,TSE,https://dadosabertos.tse.jus.br/dataset/,2.0,NaN,"[material didático, estudo independente]"


,topics,datasets
0,"[Administração Pública, Finanças e Orçamento P...","[{'data_name': 'SIGA Brasil', 'data_institutio..."
1,"[Administração Pública, Finanças e Orçamento P...","[{'data_name': 'Cota parlamentar', 'data_insti..."
2,"[Agropecuária, Alimentação, Administração Públ...",[]
3,"[Direitos Humanos, Imóveis, Habitação e Urbani...",[{'data_name': 'Mortalidade no município de Sâ...


## Testando funções

In [28]:
good_series = pd.Series(['1970-01', '1970-12', '1970-9', '2099-1', '2099-10', 
                         '01/1970', '12/1970', '9/1970', '1/2099', '10/2099',
                         ' 1970-01', '1970-12 ', ' 1970-9 ', '  2099-1  ', '2099-10', 
                         '01/1970 ', '  12/1970', '9/1970 ', '  1/2099', '10/2099  ', 
                         np.NaN, 'janeiro de 2020', '13/2000', '1800-01', '1900-01'])
good_answers = pd.Series(['1970-01', '1970-12', '1970-09', '2099-01', '2099-10', 
                          '1970-01', '1970-12', '1970-09', '2099-01', '2099-10', 
                          '1970-01', '1970-12', '1970-09', '2099-01', '2099-10', 
                          '1970-01', '1970-12', '1970-09', '2099-01', '2099-10',
                         np.NaN, np.NaN, np.NaN, np.NaN, '1900-01'])

xx.multi_test_function(std_date_series, [(good_series,)], [good_answers])

In [13]:
good_series  = pd.Series(['http://cordata.nic.br', 'https://x.com/v2/api', 'ftp://henriquexavier.net', 'did:google.com?q=abacate&t=39', 'http://www.ftp.f/aba#dsh/fvuih487&#%$@$#.html', np.NaN,
                          'htp://www.globo.com', 'www.gov.br', 'http://www efd.com', 'http://gov.br/index.html ', '   http://gov.br/index.html ', ''])
good_answers = pd.Series([False, False, False, False, False, False, True, True, True, False, False, False])

xx.multi_test_function(bad_url, [(good_series,)], [good_answers])